In [1]:
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
from datetime import timedelta

In [12]:
# Load dataset
@st.cache_data
def load_data():
    df = pd.read_csv("AAPL_stock_data.csv")
    df["Date"] = pd.to_datetime(df["date"])
    df.sort_values("Date", inplace=True)
    return df

df = load_data()
st.title("📈 Apple Stock Price Predictor")

st.subheader("📊 Raw Data")
st.write(df.tail())

2025-04-13 15:13:32.179 No runtime found, using MemoryCacheStorageManager
2025-04-13 15:13:32.200 No runtime found, using MemoryCacheStorageManager
2025-04-13 15:13:32.235 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-13 15:13:32.241 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-13 15:13:32.243 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-13 15:13:32.401 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-13 15:13:32.402 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


KeyError: 'date'

In [ ]:
# Preprocess
scaler = MinMaxScaler()
scaled_close = scaler.fit_transform(df[["Close"]])

In [ ]:
# Prepare features and targets
X = []
y = []
window_size = 60

for i in range(window_size, len(scaled_close)):
    X.append(scaled_close[i - window_size:i, 0])
    y.append(scaled_close[i, 0])

X, y = np.array(X), np.array(y)
X_lstm = X.reshape(X.shape[0], X.shape[1], 1)

In [ ]:
# Train models
st.subheader("⚙️ Training Models...")

lr_model = LinearRegression().fit(X, y)
rf_model = RandomForestRegressor(n_estimators=100).fit(X, y)

lstm_model = Sequential([
    LSTM(units=50, return_sequences=True, input_shape=(X_lstm.shape[1], 1)),
    LSTM(units=50),
    Dense(1)
])
lstm_model.compile(optimizer="adam", loss="mean_squared_error")
lstm_model.fit(X_lstm, y, epochs=5, batch_size=32, verbose=0)

st.success("Models trained!")

In [ ]:
# Choose model
model_choice = st.selectbox("Choose a model:", ["Linear Regression", "Random Forest", "LSTM"])

In [ ]:
# Predict next day
last_60 = scaled_close[-60:].reshape(1, -1)
if model_choice == "Linear Regression":
    next_scaled = lr_model.predict(last_60)
elif model_choice == "Random Forest":
    next_scaled = rf_model.predict(last_60)
else:
    next_scaled = lstm_model.predict(last_60.reshape(1, 60, 1))

next_price = scaler.inverse_transform([[next_scaled[0]]])[0][0]
next_date = df["Date"].iloc[-1] + timedelta(days=1)

st.subheader("📅 Next Day Forecast")
st.write(f"**Date:** {next_date.strftime('%Y-%m-%d')}")
st.write(f"**Predicted Close Price:** ${next_price:.2f}")

In [ ]:
# Plotting
st.subheader("📉 Last 100 Days + Forecast")
df_last = df.tail(100).copy()
df_last = df_last.append({"Date": next_date, "Close": next_price}, ignore_index=True)

fig, ax = plt.subplots()
ax.plot(df_last["Date"], df_last["Close"], marker='o')
ax.set_title("Stock Price Forecast")
ax.set_xlabel("Date")
ax.set_ylabel("Close Price")
st.pyplot(fig)